En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# **Challenge LATAM AIRLINES Data Engineer**

For this challenge a data has been given, the data is about tweets from Twitter, it comes as a JSON file wich has a size of 389Mb aprox. 

At first we have to set the file into a place it can be accessed and not in the repository, by being a large file. 

I uploaded it to `Google Cloud Storage`, inside a `bucket`. From here we'll refer to `Google Cloud Storage` as `GCS`.

The files at `GCS` has a path like `"gs://{bucket_name}/{blob.name}"`. Bucket name and file is reserved by me in my `GCS` and `env file`.

I created a `.env` file where i set my enviroment variables as
- `PROJECT_ID`: The Google Cloud Platform (`GCP`) project where the bucket of `GCS` is allowed.
- `BUCKET_NAME`: The name of the `GCS` bucket
- `GCP_CREDENTIALS`: The Json content of the key to the service account in `GCP` to log into `Google Cloud`.
- `BUCKET_FOLDER_ORIGIN`: The folder inside the bucket where data is stored.

I added a python file named `json_to_dataframe.py` where i make the connection to `GCS`. 

This file has modules to get the file path and get a dataframe from the given file.

So, now that my environment is set `I can start to code!`.

At first we will get the file path from our `cloud bucket`.

In [1]:
import json_to_dataframe as jtd 
import os
import pandas as pd
import cProfile
import pstats
from memory_profiler import profile

file_path = jtd.get_gcs_path()

Logged in to GCP / GCS client
BLOBS LISTED! ----------------------
[<Blob: latam-airlines, Data/, 1710462726242408>, <Blob: latam-airlines, Data/farmers-protest-tweets.json, 1710465820905781>]
The file name is 
The file name is farmers-protest-tweets.json
BUILDING FILE PATH! ----------------------


Now that we have the file path, we can download the file to work it offline. 

First we call from jtd the method download file, this method downloads the file into folder `src/files/`. If the file already exist in the folder, it doesn't get downloaded. The method returns the local path of the json file.

With the file downloaded, the local path file is used to read the file with `pandas` and save it to parquet format (best by compression of large data) if the parquet file doesn't exist.

Finally, build the parquet file path in local and save it in a variable.







In [2]:
directory_path = os.path.join(os.path.dirname(os.path.abspath("q1_memory.py")), "files")
local_json_file_path = jtd.download_file(file_path,directory_path)
df= pd.read_json(local_json_file_path, lines=True)
# Get the file name from the file path
file_name = os.path.basename(local_json_file_path)
# parquet file name
parquet_file_name = file_name.split('.')[0] + '.parquet'
# If parquet file does not exist, create a new parquet file
if not os.path.exists(os.path.join(directory_path, parquet_file_name)):
    df.to_parquet(os.path.join(directory_path, parquet_file_name),engine='auto')
    print("Parquet file created")
# Get the path of the parquet file
parquet_file_local_path = os.path.join(directory_path, parquet_file_name)
# Clear df from memory
del df

All questions receive as parameters a file_path and have two approaches, one **based on time** and the other one **based on memory**.

The time based approaches are optimal with the parquet files, by that reason we will pass as path parameter the parquet file path.



A better way to make better this comparison and questions is by using a ***database or data warehouse***, to execute querys. Personally, I use ***BigQuery*** because it computes querys in large remote cluster servers at the same time, which makes the time response short by dividing one Query in multiple sub-queries, optimizing memory and time at the same time. And can be consumed by reporting tools by connecting to data.

### ***Question/Challenge 1***

"Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días."

Top 10 dates where there are the most tweets. Mention the user (username) who has the most publications for each of those days.

For consulting this, we have to read the data from the file in json/parquet. 
#### **Time Approach**
I used pandas as my reader/interpreter, it returns a dataframe from the parquet file previously built.

In this module we read the parquet file using the pandas functionality .read_parquet, otherwise, i enabled the method the possibility to read json file if json file is passed as parameter.
```python
if file_path.endswith(".json"):
    df = pd.read_json(file_path, lines=True)
elif file_path.endswith(".parquet"):
    df = pd.read_parquet(file_path)
```

This approach use pandas as the reader, it reads all the file and keep it as a dataframe in memory. That makes it better for time of response to the consult, but makes data is always loaded at memory.

At first, convert the date column from datetime to format date. Takes username from user as the user.

Use `.value_counts()` to count over date field of dataframe, counts all ocurrences in the field and order it from bigger to smaller, we take the first 10 with `.head(10)`
```python
top_10_dates = df["date"].value_counts().head(10)
```

Now we have to get the user with more tweets (rows or count) on each date, by iterating on each date and using date as dataframe filter, where the user field is used in value_counts to make a count by user on the specific day. After that, take `idxmax()` to get the first row, also can be used `.head(1)`, this takes the user with most tweets. User and date are append to the python list `result`. Result returns the 10 dates with the user with most tweets in each date.

```python
result = []
# Iterate over the top 10 dates
for date in top_10_dates.index:
    # Get the user with the most tweets in the date
    user = df[df["date"] == date]["user"].value_counts().idxmax()
    # Append the date and the user to the result list
    result.append((date, user))
```

The full code of this approach is at 
[q1_time.py](https://github.com/JoRgEXx1899/latam-de-challenge/blob/main/src/q1_time.py).

In [3]:
from q1_time import q1_time

q1_t_result = q1_time(parquet_file_local_path)
print("q1_time.py executed successfully")
print("Results:")
print(q1_t_result)

q1_time.py executed successfully
Results:
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### **Memory Approach**

For this approach pandas is not used as the reader, because we have to optimize the memory use. Pandas, as i explained before, keeps data on memory which makes it not optimal for a memory optimizing approach. 


This approach iterates line by line of the JSON file, ensuring one line at time in memory.

Why not to use the parquet file? Because parquet file is focused on making better execution accros the time, but not memory.

Uses a dictionary to store by date the user count. Line by line gets the date and user of the tweet and adds +1 to the count of the user in the date.

With the dictionary built, just have to get the top user by date, then append date and top user into a list. The list is sorted according with most date count from major to minor. We slice the first 10 lines of the list and return it as the result. The full code of this approach is at [q1_memory.py](https://github.com/JoRgEXx1899/latam-de-challenge/blob/main/src/q1_memory.py).

In [4]:
from q1_memory import q1_memory

q1_m_result = q1_memory(local_json_file_path)
print("q1_memory.py executed successfully")
print("Results:")
print(q1_m_result)

q1_memory.py executed successfully
Results:
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 21), 'Surrypuria'), (datetime.date(2021, 2, 23), 'Surrypuria')]


Now that we have the result and code. Make some measures about both approaches. At first we evaluate memory approach.

In [5]:
# Load the memory_profiler extension to evaluate the memory usage in the functions
%load_ext memory_profiler
%reload_ext memory_profiler

# Evaluates the memory usage of the q1_time function
print("Memory Evaluation q1_time\n")
%memit q1_time(parquet_file_local_path)

# Evaluates the memory usage of the q1_memory function
print("\nEvaluación de memoria q1_memory\n")
print("Memory Evaluation q1_memory\n")
%memit q1_memory(local_json_file_path)


Memory Evaluation q1_time

peak memory: 1680.67 MiB, increment: 842.67 MiB

Evaluación de memoria q1_memory

Memory Evaluation q1_memory

peak memory: 1214.61 MiB, increment: 1.45 MiB


As we can see, the memory used by q1_memory is smaller than the memory used by the q1_time approach.

Now the turn for time approach. First we evaluate `q1_time` function.

In [11]:
# Instantiate the profiler
profiler = cProfile.Profile()
# Run the profiler for q1_time function
profiler.runcall(q1_time, parquet_file_local_path)
# Print the statistics of the profiler
# Get the total time spent in the function
#profiler.print_stats(sort='cumtime')
# Disable the cProfile profiler and save statistics to a file
profiler.disable()
profiler.dump_stats("output.pstats1")
# Create an instance of pstats Stats to analyze the statistics
stats = pstats.Stats("output.pstats1")
# Sort the statistics by the time spent in the function
stats.sort_stats("cumtime")
# Print the statistics
stats.print_stats(2)


Fri Mar 15 16:54:52 2024    output.pstats1

         134714 function calls (134391 primitive calls) in 2.031 seconds

   Ordered by: cumulative time
   List reduced from 617 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.003    0.003    2.031    2.031 c:\Users\danie\OneDrive\Documents\GitHub\latam-de-challenge\src\q1_time.py:7(q1_time)
        1    0.001    0.001    1.912    1.912 c:\Users\danie\.conda\envs\latamair\Lib\site-packages\pandas\io\parquet.py:428(read_parquet)




And now evaluate `q1_memory` function.

In [14]:
# Run the profiler for q1_time function
profiler.runcall(q1_memory, local_json_file_path)
# Print the statistics of the profiler
# Get the total time spent in the function
#profiler.print_stats(sort='cumtime')
# Disable the cProfile profiler and save statistics to a file
profiler.disable()
profiler.dump_stats("output.pstats1")
# Create an instance of pstats Stats to analyze the statistics
stats = pstats.Stats("output.pstats1")
# Sort the statistics by the time spent in the function
stats.sort_stats("cumtime")
# Print the statistics
stats.print_stats(2)

# Remove the pstats file
os.remove("output.pstats1")

Fri Mar 15 17:15:03 2024    output.pstats1

         9943523 function calls (9943200 primitive calls) in 15.903 seconds

   Ordered by: cumulative time
   List reduced from 642 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    2.949    0.983   13.872    4.624 c:\Users\danie\OneDrive\Documents\GitHub\latam-de-challenge\src\q1_memory.py:7(q1_memory)
   352221    5.709    0.000    5.709    0.000 {built-in method ujson.loads}




As wee can see, the `q1_time` function is better in execution time than `q1_memory` function.